In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

torch.set_printoptions(linewidth=120)
torch.set_grad_enabled(True)

from torch.utils.tensorboard import SummaryWriter

In [3]:
print(torch.__version__)
print(torchvision.__version__)

1.4.0
0.5.0


In [4]:
def get_num_correct(preds, labels):
    return preds.argmax(dim=1).eq(labels).sum().item()

In [5]:
class Network(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)
        
        self.fc1 = nn.Linear(in_features=12 *4 *4, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=60)
        self.out = nn.Linear(in_features=60, out_features=10)
        
    def forward(self, t):
        
        t = F.relu(self.conv1(t))
        t = F.max_pool2d(t, kernel_size=2, stride=2)
        
        t = F.relu(self.conv2(t))
        t = F.max_pool2d(t, kernel_size=2, stride=2)
        
        t = t.flatten(start_dim=1)
        t = F.relu(self.fc1(t))
        
        t = F.relu(self.fc2(t))
        
        t = self.out(t)
        
        return t

In [6]:
train_set = torchvision.datasets.FashionMNIST(
    root='./data',
    train=True,
    download=True,
    transform=transforms.Compose([
        transforms.ToTensor()
    ])
)

In [7]:
train_loader = torch.utils.data.DataLoader(train_set, batch_size=100, shuffle=True)

In [22]:
import numpy as np

tb = SummaryWriter()

network = Network()
images, labels = next(iter(train_loader))

grid = torchvision.utils.make_grid(images)

tb.add_image('images', grid)
tb.add_graph(network, images)
tb.close()

In [10]:
network = Network()
train_loader = torch.utils.data.DataLoader(train_set, batch_size=100, shuffle=True)
optimizer = optim.Adam(network.parameters(), lr=0.01)

images, labels = next(iter(train_loader))
grid = torchvision.utils.make_grid(images)

tb = SummaryWriter()
#tb.add_images('images', grid)
#tb.add_graph(network, images)


for epoch in range(10):
    
    total_loss = 0
    total_correct = 0
    
    for batch in train_loader:
        images, labels = batch
        
        preds = network(images)
        loss = F.cross_entropy(preds, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        total_correct += get_num_correct(preds, labels)
        
    tb.add_scalar('Loss', total_loss, epoch)
    tb.add_scalar('Number Correct', total_correct, epoch)
    tb.add_scalar('Accuracy', total_correct / len(train_set), epoch)
    
    tb.add_histogram('conv1.bias', network.conv1.bias, epoch)
    tb.add_histogram('conv1.weight', network.conv1.weight, epoch)
    tb.add_histogram('conv1.weight.grad', network.conv1.weight.grad, epoch)
        
    print("epoch", epoch, "total_correct:", total_correct, "loss", total_loss)

epoch 0 total_correct: 47462 loss 332.18918250501156
epoch 1 total_correct: 51839 loss 222.8115930557251
epoch 2 total_correct: 52510 loss 204.64502215385437
epoch 3 total_correct: 52624 loss 197.28600472956896
epoch 4 total_correct: 52987 loss 191.13311797380447
epoch 5 total_correct: 53081 loss 188.5970937386155
epoch 6 total_correct: 53203 loss 185.8166949674487
epoch 7 total_correct: 53434 loss 180.11003917455673
epoch 8 total_correct: 53539 loss 178.2805076688528
epoch 9 total_correct: 53502 loss 176.72994059324265
